In [ ]:
# 1 - exemplo do slide
from random import randint
#Cache
T0 = 'null';dado1 = [];dado0 = [] # linha 0
T1 = 'null';dado3 = [];dado2 = [] # linha 1
T2 = 'null';dado5 = [];dado4 = [] # linha 2
T3 = 'null';dado7 = [];dado6 = [] # linha 3

#MP
#B4 --> index: 12 / hex: C
#B5 --> index: 13 / hex: D

V = [[0,0,0,1,0,0,0,0,1,1,0,0], # index: 0 (1B4)--> 10C
     [0,1,0,0,0,0,0,0,1,1,0,1], # index: 1 (4B5)--> 40D
     [0,1,1,1,0,0,0,0,0,1,0,1], # index: 2 (705)--> 705
     [1,0,1,0,0,0,0,0,1,1,0,0], # index: 3 (AB4)--> A0C
     [1,0,0,0,0,0,0,0,0,1,1,0], # index: 4 (806)--> 806
     [1,0,1,0,0,0,0,0,1,1,0,1], # index: 5 (AB5)--> A0D
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 6  000
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,1,0,1,0,1,1,0,0,1,1], # index: 12 2B3
     [0,0,0,1,1,0,1,0,0,0,1,1], # index: 13 1A3
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0]]

contador = 1
# Partes do processador:

CI = [0,0,0,0,0,0,0,0] #(CI <-- CI + 1)
R0 = [0,0,0,0,0,0,0,0,0,0,0,0]

EOp = [0,0,0,0,0,0,0,0]
COp = [0,0,0,0]
Op  = [0,0,0,0,0,0,0,0,0,0,0,0,]

# funções:
def ListToDecimalInt(x): # recebe uma lista de binários e retornar seu valor em decimal
    stri = ''
    if x[0] == '1': # número negativo
        x.pop(0)
        for i in x:
            s = str(i)
            stri = stri + s
        int1 = (int(stri,2))
        return -int1
    else:          # número positivo
        for i in x: 
            s = str(i)
            stri = stri + s
        int1 = (int(stri,2))
        return int1

def DecimalIntToList(x): # recebe um decimal e retorna uma lista em binário
    list1 = []
    comp2 = ''
    binary = bin(x).replace("0b", "")
    if binary[0] == '-': # número negativo
        binary.replace("-","")
        for i in binary:
            if i == '0':
                comp2 = comp2 + '1'
            elif i == '1':
                comp2 = comp2 + '0'
        soma = bin(int(comp2,2) + 1).replace("0b", "")
        for i in soma:
            int1 = int(i)
            list1.append(int1)
        while len(list1)<12:
            list1.insert(0,0)
        list1.insert(0,'1')
        return list1
    else:
        for i in binary: # número positivo
            int1 = int(i)
            list1.append(int1)
        while len(list1) < 12:
            list1.insert(0,0)
        return list1
    
#--------------- CI(c) , EOp(w) , COp(x) , Op(y) e R0(z) -----------------------#
    
def operacaoOP(x,z,w): # operações que retornam o Op
    if x == [0,0,1,0]:
        w = ListToDecimalInt(w)
        V.pop(w)
        V.insert(w,z)
        print('operação: Store')
        print('armazenando:',z)

def operacaoR0(x,z,y): # operações que retornam o R0
    if x == [0,0,0,1]:
        z = y
        while len(z) < 12:
            z.insert(0,0)
        print('operação: Load')
        print('novo R0:',z)
        return z
    
    elif x == [0,0,1,1]:
        print('operação: ADD')
        print('R0:',z,'+','Op:',y)
        z = ListToDecimalInt(z)
        y = ListToDecimalInt(y)
        total = z + y
        total = DecimalIntToList(total)
        z = total
        while len(z) < 12:
            z.insert(0,0)
        print('resultado:',z)
        return z
    
    elif x == [0,1,0,0]:
        print('operação: SUB | R0<-- R0 -(Op)|')
        print('R0:',z,'-','Op:',y)
        z = ListToDecimalInt(z)
        y = ListToDecimalInt(y)
        total = z - y
        total = DecimalIntToList(total)
        z = total
        while len(z) < 12:
            z.insert(0,0)
        print('resultado:',z)
        return z

    else:
        return z

def operacaoCI(x,c,w,z,y):# operações que retornam o CI + get e PRT
    if x == [0,1,0,1]:
        print('operação: JZ')
        if R0 == [0,0,0,0,0,0,0,0,0,0,0,0]:
            c = w
            return c
        else:
            print('operação pulada --> R0 != 0')
            return c
    elif x == [0,1,1,0]:
        print('operação: JP')
        conta = ListToDecimalInt(z)
        if conta > 0:
            c = w
            return c
        else:
            print('operação pulada --> R0 não é maior que zero')
            return c

    elif x == [0,1,1,1]:
        print('operação: JN')
        conta = ListToDecimalInt(z)
        if conta < 0:
            c = w
            return c
        else:
            print('operação pulada --> R0 não é menor que zero')
            return c

    elif x == [1,0,0,0]:
        c = w
        print('operação: JUMP')
        return c
    
    elif x == [1,0,0,1]:
        print('operação: GET')
        return c
        
    elif x == [1,0,1,0]:
        print('operação: PRT')
        print('Op:',y)
        return c

    else:
        return c

def CampoOperacao(x):  # resgata o código de operação na instrução (recebe instrucao)
    COp = x[:4]
    print('COp: ',COp)
    return COp

def CampoOperador(x):  # resgata o endereço do operando/instrução na instrução (recebe instrucao)
    EOp = x[4:]
    print('EOp: ',EOp)
    return EOp

def DadoOperador(x):   # retorna o próprio operando/intrução (recebe EOp)
    decimal = ListToDecimalInt(x)
    Op = V[decimal]
    print('Op:  ',Op)
    return Op



# partes da cache:
LargTag = 2
LargConjunto = 1 
LargCelula = 1
print('cache:')
print('largura da tag: 2 bits')
print('largura do conjunto: 1 bit')
print('largura da célula: 1 bit\n\n')

while True:
    I = input('Digite E para escrever, L para Ler, T para listar toda memoria principal, C para iniciar o ciclo de instrução e qualquer tecla para parar:')
    if I == 'E':
        # parte de escrita da cache:
        END = input('\ndigite um endereço de 4 bits:')
        campoTag = END[:LargTag]
        campoConjunto = END[LargTag:LargTag + LargConjunto]
        campoCelula = END[LargTag + LargConjunto:]
        bloco = END[:LargTag + LargConjunto]
        print('endereço do bloco:',bloco,'\n')
        celula = END[LargTag + LargConjunto:]
        print('endereço da célula:',celula,'\n')
        Nbloco = int(bloco,2)
        Ncelula = int(celula,2)
        d = Ncelula + 2 * Nbloco
        print('campoTag:',campoTag)
        print('campoConjunto:',campoConjunto)
        print('campoCélula:',campoCelula,'\n')
        V.pop(d)
        a = int(input('Digite o dado de 12 bits:'))
        l = list(str(a))
        while len(l)<12:
            l.insert(0,0)
            
        # escrita no conjunto 0
        if campoConjunto == '0':
            RN = randint(0,9)
            if RN % 2 == 0:
                T0 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 0, célula 0:\n')
                    dado0 = l
                    dado1 = V[d+1]
                    V.insert(d,dado0)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 0, célula 1:\n')
                    dado1 = l
                    dado0 = V[d-1]
                    V.insert(d,dado1)
            elif RN % 2 == 1:
                T1 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 1, célula 2:\n')
                    dado2 = l
                    dado3 = V[d+1]
                    V.insert(d,dado2)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 1, célula 3:\n')
                    dado3 = l
                    dado2 = V[d-1]
                    V.insert(d,dado3)
        # escrita no conjunto 1
        elif campoConjunto == '1':
            RN = randint(0,9)
            if RN % 2 == 0:
                T2 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 2, célula 4:\n')
                    dado4 = l
                    dado5 = V[d+1]
                    V.insert(d,dado4)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 2, célula 5:\n')
                    dado5 = l
                    dado4 = V[d-1]
                    V.insert(d,dado5)
            elif RN % 2 == 1:
                T3 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 3, célula 6:\n')
                    dado6 = l
                    dado7 = V[d+1]
                    V.insert(d,dado6)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 3, célula 7:\n')
                    dado7 = l
                    dado6 = V[d-1]
                    V.insert(d,dado7)    
    elif I == 'L':
        # parte de leitura da cache:
        END = input('\ndigite um endereço de 4 bits:')
        campoTag = END[:LargTag]
        campoConjunto = END[LargTag:LargTag + LargConjunto]
        campoCelula = END[LargTag + LargConjunto:]
        bloco = END[:LargTag + LargConjunto]
        print('endereço do bloco:',bloco)
        celula = END[LargTag + LargConjunto:]
        print('endereço da célula:',celula)
        Nbloco = int(bloco,2)
        Ncelula = int(celula,2)
        d = Ncelula + 2 * Nbloco
        print('campoTag:',campoTag)
        print('campoConjunto:',campoConjunto)
        print('campoCélula:',campoCelula)
        # leitura no conjunto 0
        if campoConjunto == '0':
            if T0 == campoTag:
                print('--hit linha 0--')
                if campoCelula == '0':
                    print('celula 0:',dado0)
                elif campoCelula == '1':
                    print('celula 1:',dado1)
            elif T1 == campoTag:
                print('--hit linha 1--')
                if campoCelula == '0':
                    print('celula 2:',dado2)
                elif campoCelula == '1':
                    print('celula 3:',dado3)
            else:
                print('\n--miss--\n')
                print('política de substituição de linha atuando:')
                RN = randint(0,9)
                if RN % 2 == 0:
                    print('substituindo a linha 0\n')
                    T0 = campoTag
                    dado0 = V[Nbloco + Nbloco]
                    dado1 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada da MP para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado0)
                    elif campoCelula == '1':
                        print('célula 1:',dado1)
                elif RN % 2 == 1:
                    print('substituindo a linha 1\n')
                    T1 = campoTag
                    dado2 = V[Nbloco + Nbloco]
                    dado3 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 2:',dado2)
                    elif campoCelula == '1':
                        print('célula 3:',dado3)
                        
                # leitura no conjunto 1
        elif campoConjunto == '1':
            if T2 == campoTag:
                print('--hit linha 2--')
                if campoCelula == '0':
                    print('célula 0:',dado4)
                elif campoCelula == '1':
                    print('célula 1:',dado5)
            elif T3 == campoTag:
                print('--hit linha 3--')
                if campoCelula == '0':
                    print('célula 0:',dado6)
                elif campoCelula == '1':
                    print('célula 1:',dado7)
            else:
                print('\n--miss--\n')
                print('atuando sobre a política de substituição de linha:')
                RN = randint(0,9)
                if RN % 2 == 0:
                    print('substituindo a linha 2\n')
                    T2 = campoTag
                    dado4 = V[Nbloco + Nbloco]
                    dado5 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado4)
                    elif campoCelula == '1':
                        print('célula 1:',dado5)
                elif RN % 2 == 1:
                    print('substituindo a linha 3\n')
                    T3 = campoTag
                    dado6 = V[Nbloco + Nbloco]
                    dado7 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado6)
                    elif campoCelula == '1':
                        print('célula 1:',dado7)
    elif I == 'T':
        for i in range(len(V)):
            print(*V[i], sep='')
            
    elif I == 'C':
        while True:
            print()
            print('ciclo:',contador)
            print('Registrador CI:',CI)
    
            CI = ListToDecimalInt(CI)
            instrucao = V[CI]
    
            print('instrução:',instrucao)
            COp = CampoOperacao(instrucao)
            while len(COp) > 4:
                COp.insert(0,0)
            EOp = CampoOperador(instrucao)
            Op = DadoOperador(EOp)

            CI = DecimalIntToList(CI)
            CI = CI[4:]

            # parte da UAL
    
            if COp == [0,0,0,0]:
                CI = [0,0,0,0,0,0,0,0]
                print('operação: Halt')
                break ## continue
    
            elif COp == [0,0,1,0]:
                operacaoOP(COp,R0,EOp)
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]
        
            elif COp == [0,0,0,1] or COp == [0,0,1,1] or COp == [0,1,0,0]:
                R0 = operacaoR0(COp,R0,Op)
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]

        
            else:
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]
                CI = operacaoCI(COp,CI,EOp,R0,Op)

            contador += 1
    else:
        break


In [ ]:
# 2 - de 10 a 0
from random import randint
#Cache
T0 = 'null';dado1 = [];dado0 = [] # linha 0
T1 = 'null';dado3 = [];dado2 = [] # linha 1
T2 = 'null';dado5 = [];dado4 = [] # linha 2
T3 = 'null';dado7 = [];dado6 = [] # linha 3

#MP
#B4 --> index: 12 / hex: C
#B5 --> index: 13 / hex: D

V = [[1,0,1,0,0,0,0,0,1,1,0,1], # index: 0 (AB5)--> A0D 
     [0,0,0,1,0,0,0,0,1,1,0,1], # index: 1 (1B5)--> 10D 
     [0,1,0,0,0,0,0,0,1,1,0,0], # index: 2 (4B4)--> 40C 
     [0,0,1,0,0,0,0,0,1,1,0,1], # index: 3 (2B5)--> 20D 
     [1,0,1,0,0,0,0,0,1,1,0,1], # index: 4 (AB5)--> A0D 
     [0,1,0,1,0,0,0,0,0,1,1,1], # index: 5 (507)--> 507 
     [1,0,0,0,0,0,0,0,0,0,0,1], # index: 6 (801)--> 801
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 7  000 
     [1,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,1], # index: (B4) 12 001 --> 001
     [0,0,0,0,0,0,0,0,1,0,1,0], # index: (B5) 13 010 --> 00A
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0]]

contador = 1
# Partes do processador:

CI = [0,0,0,0,0,0,0,0] #(CI <-- CI + 1)
R0 = [0,0,0,0,0,0,0,0,0,0,0,0]

EOp = [0,0,0,0,0,0,0,0]
COp = [0,0,0,0]
Op  = [0,0,0,0,0,0,0,0,0,0,0,0,]

# funções:
def ListToDecimalInt(x): # recebe uma lista de binários e retornar seu valor em decimal
    stri = ''
    if x[0] == '1': # número negativo
        x.pop(0)
        for i in x:
            s = str(i)
            stri = stri + s
        int1 = (int(stri,2))
        return -int1
    else:          # número positivo
        for i in x: 
            s = str(i)
            stri = stri + s
        int1 = (int(stri,2))
        return int1

def DecimalIntToList(x): # recebe um decimal e retorna uma lista em binário
    list1 = []
    comp2 = ''
    binary = bin(x).replace("0b", "")
    if binary[0] == '-': # número negativo
        binary.replace("-","")
        for i in binary:
            if i == '0':
                comp2 = comp2 + '1'
            elif i == '1':
                comp2 = comp2 + '0'
        soma = bin(int(comp2,2) + 1).replace("0b", "")
        for i in soma:
            int1 = int(i)
            list1.append(int1)
        while len(list1)<12:
            list1.insert(0,0)
        list1.insert(0,'1')
        return list1
    else:
        for i in binary: # número positivo
            int1 = int(i)
            list1.append(int1)
        while len(list1) < 12:
            list1.insert(0,0)
        return list1
    
#--------------- CI(c) , EOp(w) , COp(x) , Op(y) e R0(z) -----------------------#
    
def operacaoOP(x,z,w): # operações que retornam o Op
    if x == [0,0,1,0]:
        w = ListToDecimalInt(w)
        V.pop(w)
        V.insert(w,z)
        print('operação: Store')
        print('armazenando:',z)

def operacaoR0(x,z,y): # operações que retornam o R0
    if x == [0,0,0,1]:
        z = y
        while len(z) < 12:
            z.insert(0,0)
        print('operação: Load')
        print('novo R0:',z)
        return z
    
    elif x == [0,0,1,1]:
        print('operação: ADD')
        print('R0:',z,'+','Op:',y)
        z = ListToDecimalInt(z)
        y = ListToDecimalInt(y)
        total = z + y
        total = DecimalIntToList(total)
        z = total
        while len(z) < 12:
            z.insert(0,0)
        print('resultado:',z)
        return z
    
    elif x == [0,1,0,0]:
        print('operação: SUB | R0<-- R0 -(Op)|')
        print('R0:',z,'-','Op:',y)
        z = ListToDecimalInt(z)
        y = ListToDecimalInt(y)
        total = z - y
        total = DecimalIntToList(total)
        z = total
        while len(z) < 12:
            z.insert(0,0)
        print('resultado:',z)
        return z

    else:
        return z

def operacaoCI(x,c,w,z,y):# operações que retornam o CI + get e PRT
    if x == [0,1,0,1]:
        print('operação: JZ')
        if R0 == [0,0,0,0,0,0,0,0,0,0,0,0]:
            c = w
            return c
        else:
            print('operação pulada --> R0 != 0')
            return c
    elif x == [0,1,1,0]:
        print('operação: JP')
        conta = ListToDecimalInt(z)
        if conta > 0:
            c = w
            return c
        else:
            print('operação pulada --> R0 não é maior que zero')
            return c

    elif x == [0,1,1,1]:
        print('operação: JN')
        conta = ListToDecimalInt(z)
        if conta < 0:
            c = w
            return c
        else:
            print('operação pulada --> R0 não é menor que zero')
            return c

    elif x == [1,0,0,0]:
        c = w
        print('operação: JUMP')
        return c
    
    elif x == [1,0,0,1]:
        print('operação: GET')
        return c
        
    elif x == [1,0,1,0]:
        print('operação: PRT')
        print('Op:',y)
        return c

    else:
        return c

def CampoOperacao(x):  # resgata o código de operação na instrução (recebe instrucao)
    COp = x[:4]
    print('COp: ',COp)
    return COp

def CampoOperador(x):  # resgata o endereço do operando/instrução na instrução (recebe instrucao)
    EOp = x[4:]
    print('EOp: ',EOp)
    return EOp

def DadoOperador(x):   # retorna o próprio operando/intrução (recebe EOp)
    decimal = ListToDecimalInt(x)
    Op = V[decimal]
    print('Op:  ',Op)
    return Op



# partes da cache:
LargTag = 2
LargConjunto = 1 
LargCelula = 1
print('cache:')
print('largura da tag: 2 bits')
print('largura do conjunto: 1 bit')
print('largura da célula: 1 bit\n\n')

while True:
    I = input('Digite E para escrever, L para Ler, T para listar toda memoria principal, C para iniciar o ciclo de instrução e qualquer tecla para parar:')
    if I == 'E':
        # parte de escrita da cache:
        END = input('\ndigite um endereço de 4 bits:')
        campoTag = END[:LargTag]
        campoConjunto = END[LargTag:LargTag + LargConjunto]
        campoCelula = END[LargTag + LargConjunto:]
        bloco = END[:LargTag + LargConjunto]
        print('endereço do bloco:',bloco,'\n')
        celula = END[LargTag + LargConjunto:]
        print('endereço da célula:',celula,'\n')
        Nbloco = int(bloco,2)
        Ncelula = int(celula,2)
        d = Ncelula + 2 * Nbloco
        print('campoTag:',campoTag)
        print('campoConjunto:',campoConjunto)
        print('campoCélula:',campoCelula,'\n')
        V.pop(d)
        a = int(input('Digite o dado de 12 bits:'))
        l = list(str(a))
        while len(l)<12:
            l.insert(0,0)
            
        # escrita no conjunto 0
        if campoConjunto == '0':
            RN = randint(0,9)
            if RN % 2 == 0:
                T0 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 0, célula 0:\n')
                    dado0 = l
                    dado1 = V[d+1]
                    V.insert(d,dado0)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 0, célula 1:\n')
                    dado1 = l
                    dado0 = V[d-1]
                    V.insert(d,dado1)
            elif RN % 2 == 1:
                T1 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 1, célula 2:\n')
                    dado2 = l
                    dado3 = V[d+1]
                    V.insert(d,dado2)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 1, célula 3:\n')
                    dado3 = l
                    dado2 = V[d-1]
                    V.insert(d,dado3)
        # escrita no conjunto 1
        elif campoConjunto == '1':
            RN = randint(0,9)
            if RN % 2 == 0:
                T2 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 2, célula 4:\n')
                    dado4 = l
                    dado5 = V[d+1]
                    V.insert(d,dado4)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 2, célula 5:\n')
                    dado5 = l
                    dado4 = V[d-1]
                    V.insert(d,dado5)
            elif RN % 2 == 1:
                T3 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 3, célula 6:\n')
                    dado6 = l
                    dado7 = V[d+1]
                    V.insert(d,dado6)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 3, célula 7:\n')
                    dado7 = l
                    dado6 = V[d-1]
                    V.insert(d,dado7)    
    elif I == 'L':
        # parte de leitura da cache:
        END = input('\ndigite um endereço de 4 bits:')
        campoTag = END[:LargTag]
        campoConjunto = END[LargTag:LargTag + LargConjunto]
        campoCelula = END[LargTag + LargConjunto:]
        bloco = END[:LargTag + LargConjunto]
        print('endereço do bloco:',bloco)
        celula = END[LargTag + LargConjunto:]
        print('endereço da célula:',celula)
        Nbloco = int(bloco,2)
        Ncelula = int(celula,2)
        d = Ncelula + 2 * Nbloco
        print('campoTag:',campoTag)
        print('campoConjunto:',campoConjunto)
        print('campoCélula:',campoCelula)
        # leitura no conjunto 0
        if campoConjunto == '0':
            if T0 == campoTag:
                print('--hit linha 0--')
                if campoCelula == '0':
                    print('celula 0:',dado0)
                elif campoCelula == '1':
                    print('celula 1:',dado1)
            elif T1 == campoTag:
                print('--hit linha 1--')
                if campoCelula == '0':
                    print('celula 2:',dado2)
                elif campoCelula == '1':
                    print('celula 3:',dado3)
            else:
                print('\n--miss--\n')
                print('política de substituição de linha atuando:')
                RN = randint(0,9)
                if RN % 2 == 0:
                    print('substituindo a linha 0\n')
                    T0 = campoTag
                    dado0 = V[Nbloco + Nbloco]
                    dado1 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada da MP para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado0)
                    elif campoCelula == '1':
                        print('célula 1:',dado1)
                elif RN % 2 == 1:
                    print('substituindo a linha 1\n')
                    T1 = campoTag
                    dado2 = V[Nbloco + Nbloco]
                    dado3 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 2:',dado2)
                    elif campoCelula == '1':
                        print('célula 3:',dado3)
                        
                # leitura no conjunto 1
        elif campoConjunto == '1':
            if T2 == campoTag:
                print('--hit linha 2--')
                if campoCelula == '0':
                    print('célula 0:',dado4)
                elif campoCelula == '1':
                    print('célula 1:',dado5)
            elif T3 == campoTag:
                print('--hit linha 3--')
                if campoCelula == '0':
                    print('célula 0:',dado6)
                elif campoCelula == '1':
                    print('célula 1:',dado7)
            else:
                print('\n--miss--\n')
                print('atuando sobre a política de substituição de linha:')
                RN = randint(0,9)
                if RN % 2 == 0:
                    print('substituindo a linha 2\n')
                    T2 = campoTag
                    dado4 = V[Nbloco + Nbloco]
                    dado5 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado4)
                    elif campoCelula == '1':
                        print('célula 1:',dado5)
                elif RN % 2 == 1:
                    print('substituindo a linha 3\n')
                    T3 = campoTag
                    dado6 = V[Nbloco + Nbloco]
                    dado7 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado6)
                    elif campoCelula == '1':
                        print('célula 1:',dado7)
    elif I == 'T':
        for i in range(len(V)):
            print(*V[i], sep='')
            
    elif I == 'C':
        while True:
            print()
            print('ciclo:',contador)
            print('Registrador CI:',CI)
    
            CI = ListToDecimalInt(CI)
            instrucao = V[CI]
    
            print('instrução:',instrucao)
            COp = CampoOperacao(instrucao)
            while len(COp) > 4:
                COp.insert(0,0)
            EOp = CampoOperador(instrucao)
            Op = DadoOperador(EOp)

            CI = DecimalIntToList(CI)
            CI = CI[4:]

            # parte da UAL
    
            if COp == [0,0,0,0]:
                CI = [0,0,0,0,0,0,0,0]
                print('operação: Halt')
                break ## continue
    
            elif COp == [0,0,1,0]:
                operacaoOP(COp,R0,EOp) 
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]
        
            elif COp == [0,0,0,1] or COp == [0,0,1,1] or COp == [0,1,0,0]:
                R0 = operacaoR0(COp,R0,Op)
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]

        
            else:
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]
                CI = operacaoCI(COp,CI,EOp,R0,Op)

            # parte do incremento do CI
            '''
            CI = ListToDecimalInt(CI)
            CI += 1
            CI = DecimalIntToList(CI)
            CI = CI[4:]  
            contador += 1
            '''
            contador += 1
    else:
        break


In [ ]:
# 3 - mostrar o maior entre os dois
from random import randint
#Cache
T0 = 'null';dado1 = [];dado0 = [] # linha 0
T1 = 'null';dado3 = [];dado2 = [] # linha 1
T2 = 'null';dado5 = [];dado4 = [] # linha 2
T3 = 'null';dado7 = [];dado6 = [] # linha 3

#MP
#B4 --> index: 12 / hex: C
#B5 --> index: 13 / hex: D

V = [[0,0,0,1,0,0,0,0,1,1,0,0], # index: 0 (1B4)--> 10C  load(B4)
     [0,1,0,0,0,0,0,0,1,1,0,1], # index: 1 (4B5)--> 40D  sub(R0 - Op(B5))
     [0,1,1,0,0,0,0,0,0,1,1,0], # index: 2 (606)         jump positive(06)
     [0,1,0,1,0,0,0,0,1,0,0,0], # index: 3 (508)         jump zero(08)
     [1,0,1,0,0,0,0,0,1,1,0,1], # index: 4 (AB5)--> A0D  print(B5)
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 5 (000)         halt
     [1,0,1,0,0,0,0,0,1,1,0,0], # index: 6 (AB4)--> A0C  print(B4)
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 7 (000)         halt
     [1,0,1,0,0,0,0,0,1,1,1,0], # index: 8 (AB6)--> A0E  print(B6)
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 9 (000)         halt
     [0,0,0,0,0,0,0,0,0,0,0,0], 
     [0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 12/B4 (C) 
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 13/B5 (D) 
     [0,0,0,0,0,0,0,0,0,0,0,0], # index: 14/B6 (E) por padrão --> 000
     [0,0,0,0,0,0,0,0,0,0,0,0]]

contador = 1
# Partes do processador:

CI = [0,0,0,0,0,0,0,0] #(CI <-- CI + 1)
R0 = [0,0,0,0,0,0,0,0,0,0,0,0]

EOp = [0,0,0,0,0,0,0,0]
COp = [0,0,0,0]
Op  = [0,0,0,0,0,0,0,0,0,0,0,0,]

# funções:
def ListToDecimalInt(x): # recebe uma lista de binários e retornar seu valor em decimal
    stri = ''
    if x[0] == '1': # número negativo
        x.pop(0)
        for i in x:
            s = str(i)
            stri = stri + s
        int1 = (int(stri,2))
        return -int1
    else:          # número positivo
        for i in x: 
            s = str(i)
            stri = stri + s
        int1 = (int(stri,2))
        return int1

def DecimalIntToList(x): # recebe um decimal e retorna uma lista em binário
    list1 = []
    comp2 = ''
    binary = bin(x).replace("0b", "")
    if binary[0] == '-': # número negativo
        binary.replace("-","")
        for i in binary:
            if i == '0':
                comp2 = comp2 + '1'
            elif i == '1':
                comp2 = comp2 + '0'
        soma = bin(int(comp2,2) + 1).replace("0b", "")
        for i in soma:
            int1 = int(i)
            list1.append(int1)
        while len(list1)<12:
            list1.insert(0,0)
        list1.insert(0,'1')
        return list1
    else:
        for i in binary: # número positivo
            int1 = int(i)
            list1.append(int1)
        while len(list1) < 12:
            list1.insert(0,0)
        return list1
    
#--------------- CI(c) , EOp(w) , COp(x) , Op(y) e R0(z) -----------------------#
    
def operacaoOP(x,z,w): # operações que retornam o Op
    if x == [0,0,1,0]:
        w = ListToDecimalInt(w)
        V.pop(w)
        V.insert(w,z)
        print('operação: Store')
        print('armazenando:',z)

def operacaoR0(x,z,y): # operações que retornam o R0
    if x == [0,0,0,1]:
        z = y
        while len(z) < 12:
            z.insert(0,0)
        print('operação: Load')
        print('novo R0:',z)
        return z
    
    elif x == [0,0,1,1]:
        print('operação: ADD')
        print('R0:',z,'+','Op:',y)
        z = ListToDecimalInt(z)
        y = ListToDecimalInt(y)
        total = z + y
        total = DecimalIntToList(total)
        z = total
        while len(z) < 12:
            z.insert(0,0)
        print('resultado:',z)
        return z
    
    elif x == [0,1,0,0]:
        print('operação: SUB | R0<-- R0 -(Op)|')
        print('R0:',z,'-','Op:',y)
        z = ListToDecimalInt(z)
        y = ListToDecimalInt(y)
        total = z - y
        total = DecimalIntToList(total)
        z = total
        while len(z) < 12:
            z.insert(0,0)
        print('resultado:',z)
        return z

    else:
        return z

def operacaoCI(x,c,w,z,y):# operações que retornam o CI + get e PRT
    if x == [0,1,0,1]:
        print('operação: JZ')
        if R0 == [0,0,0,0,0,0,0,0,0,0,0,0]:
            c = w
            return c
        else:
            print('operação pulada --> R0 != 0')
            return c
    elif x == [0,1,1,0]:
        print('operação: JP')
        conta = ListToDecimalInt(z)
        if conta > 0:
            c = w
            return c
        else:
            print('operação pulada --> R0 não é maior que zero')
            return c

    elif x == [0,1,1,1]:
        print('operação: JN')
        conta = ListToDecimalInt(z)
        if conta < 0:
            c = w
            return c
        else:
            print('operação pulada --> R0 não é menor que zero')
            return c

    elif x == [1,0,0,0]:
        c = w
        print('operação: JUMP')
        return c
    
    elif x == [1,0,0,1]:
        print('operação: GET')
        return c
        
    elif x == [1,0,1,0]:
        print('operação: PRT')
        print('Op:',y)
        return c

    else:
        return c

def CampoOperacao(x):  # resgata o código de operação na instrução (recebe instrucao)
    COp = x[:4]
    print('COp: ',COp)
    return COp

def CampoOperador(x):  # resgata o endereço do operando/instrução na instrução (recebe instrucao)
    EOp = x[4:]
    print('EOp: ',EOp)
    return EOp

def DadoOperador(x):   # retorna o próprio operando/intrução (recebe EOp)
    decimal = ListToDecimalInt(x)
    Op = V[decimal]
    print('Op:  ',Op)
    return Op



# partes da cache:
LargTag = 2
LargConjunto = 1 
LargCelula = 1
print('cache:')
print('largura da tag: 2 bits')
print('largura do conjunto: 1 bit')
print('largura da célula: 1 bit\n\n')

while True:
    I = input('Digite E para escrever, L para Ler, T para listar toda memoria principal, C para iniciar o ciclo de instrução e qualquer tecla para parar:')
    if I == 'E':
        # parte de escrita da cache:
        END = input('\ndigite um endereço de 4 bits:')
        campoTag = END[:LargTag]
        campoConjunto = END[LargTag:LargTag + LargConjunto]
        campoCelula = END[LargTag + LargConjunto:]
        bloco = END[:LargTag + LargConjunto]
        print('endereço do bloco:',bloco,'\n')
        celula = END[LargTag + LargConjunto:]
        print('endereço da célula:',celula,'\n')
        Nbloco = int(bloco,2)
        Ncelula = int(celula,2)
        d = Ncelula + 2 * Nbloco
        print('campoTag:',campoTag)
        print('campoConjunto:',campoConjunto)
        print('campoCélula:',campoCelula,'\n')
        V.pop(d)
        a = int(input('Digite o dado de 12 bits:'))
        l = list(str(a))
        while len(l)<12:
            l.insert(0,0)
            
        # escrita no conjunto 0
        if campoConjunto == '0':
            RN = randint(0,9)
            if RN % 2 == 0:
                T0 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 0, célula 0:\n')
                    dado0 = l
                    dado1 = V[d+1]
                    V.insert(d,dado0)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 0, célula 1:\n')
                    dado1 = l
                    dado0 = V[d-1]
                    V.insert(d,dado1)
            elif RN % 2 == 1:
                T1 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 1, célula 2:\n')
                    dado2 = l
                    dado3 = V[d+1]
                    V.insert(d,dado2)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 1, célula 3:\n')
                    dado3 = l
                    dado2 = V[d-1]
                    V.insert(d,dado3)
        # escrita no conjunto 1
        elif campoConjunto == '1':
            RN = randint(0,9)
            if RN % 2 == 0:
                T2 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 2, célula 4:\n')
                    dado4 = l
                    dado5 = V[d+1]
                    V.insert(d,dado4)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 2, célula 5:\n')
                    dado5 = l
                    dado4 = V[d-1]
                    V.insert(d,dado5)
            elif RN % 2 == 1:
                T3 = campoTag
                if campoCelula == '0':
                    print('sobrescrevendo a linha 3, célula 6:\n')
                    dado6 = l
                    dado7 = V[d+1]
                    V.insert(d,dado6)
                elif campoCelula == '1':
                    print('sobrescrevendo a linha 3, célula 7:\n')
                    dado7 = l
                    dado6 = V[d-1]
                    V.insert(d,dado7)    
    elif I == 'L':
        # parte de leitura da cache:
        END = input('\ndigite um endereço de 4 bits:')
        campoTag = END[:LargTag]
        campoConjunto = END[LargTag:LargTag + LargConjunto]
        campoCelula = END[LargTag + LargConjunto:]
        bloco = END[:LargTag + LargConjunto]
        print('endereço do bloco:',bloco)
        celula = END[LargTag + LargConjunto:]
        print('endereço da célula:',celula)
        Nbloco = int(bloco,2)
        Ncelula = int(celula,2)
        d = Ncelula + 2 * Nbloco
        print('campoTag:',campoTag)
        print('campoConjunto:',campoConjunto)
        print('campoCélula:',campoCelula)
        # leitura no conjunto 0
        if campoConjunto == '0':
            if T0 == campoTag:
                print('--hit linha 0--')
                if campoCelula == '0':
                    print('celula 0:',dado0)
                elif campoCelula == '1':
                    print('celula 1:',dado1)
            elif T1 == campoTag:
                print('--hit linha 1--')
                if campoCelula == '0':
                    print('celula 2:',dado2)
                elif campoCelula == '1':
                    print('celula 3:',dado3)
            else:
                print('\n--miss--\n')
                print('política de substituição de linha atuando:')
                RN = randint(0,9)
                if RN % 2 == 0:
                    print('substituindo a linha 0\n')
                    T0 = campoTag
                    dado0 = V[Nbloco + Nbloco]
                    dado1 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada da MP para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado0)
                    elif campoCelula == '1':
                        print('célula 1:',dado1)
                elif RN % 2 == 1:
                    print('substituindo a linha 1\n')
                    T1 = campoTag
                    dado2 = V[Nbloco + Nbloco]
                    dado3 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 2:',dado2)
                    elif campoCelula == '1':
                        print('célula 3:',dado3)
                        
                # leitura no conjunto 1
        elif campoConjunto == '1':
            if T2 == campoTag:
                print('--hit linha 2--')
                if campoCelula == '0':
                    print('célula 0:',dado4)
                elif campoCelula == '1':
                    print('célula 1:',dado5)
            elif T3 == campoTag:
                print('--hit linha 3--')
                if campoCelula == '0':
                    print('célula 0:',dado6)
                elif campoCelula == '1':
                    print('célula 1:',dado7)
            else:
                print('\n--miss--\n')
                print('atuando sobre a política de substituição de linha:')
                RN = randint(0,9)
                if RN % 2 == 0:
                    print('substituindo a linha 2\n')
                    T2 = campoTag
                    dado4 = V[Nbloco + Nbloco]
                    dado5 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado4)
                    elif campoCelula == '1':
                        print('célula 1:',dado5)
                elif RN % 2 == 1:
                    print('substituindo a linha 3\n')
                    T3 = campoTag
                    dado6 = V[Nbloco + Nbloco]
                    dado7 = V[Nbloco + Nbloco + 1]
                    print('recuperando e armazenando bloco na cache:')
                    print('célula recuperada para leitura:')
                    if campoCelula == '0':
                        print('célula 0:',dado6)
                    elif campoCelula == '1':
                        print('célula 1:',dado7)
    elif I == 'T':
        for i in range(len(V)):
            print(*V[i], sep='')
            
    elif I == 'C':
        while True:
            print()
            print('ciclo:',contador)
            print('Registrador CI:',CI)
    
            CI = ListToDecimalInt(CI)
            instrucao = V[CI]
    
            print('instrução:',instrucao)
            COp = CampoOperacao(instrucao)
            while len(COp) > 4:
                COp.insert(0,0)
            EOp = CampoOperador(instrucao)
            Op = DadoOperador(EOp)

            CI = DecimalIntToList(CI)
            CI = CI[4:]

            # parte da UAL
    
            if COp == [0,0,0,0]:
                CI = [0,0,0,0,0,0,0,0]
                print('operação: Halt')
                break ## continue
    
            elif COp == [0,0,1,0]:
                operacaoOP(COp,R0,EOp) 
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]
        
            elif COp == [0,0,0,1] or COp == [0,0,1,1] or COp == [0,1,0,0]:
                R0 = operacaoR0(COp,R0,Op)
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]

        
            else:
                CI = ListToDecimalInt(CI)
                CI += 1
                CI = DecimalIntToList(CI)
                CI = CI[4:]
                CI = operacaoCI(COp,CI,EOp,R0,Op)

            contador += 1
    else:
        break